In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug  8 19:31:34 2019

@author: mor
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask
import dask.array as da
from dask import delayed
import time
import dask.dataframe as dd
import dask.bag as db
import glob
import json


In [2]:

# Glob filenames matching 'sotu/*.txt' and sort
filenames = glob.glob('data\\sotu\\*.txt')
filenames = sorted(filenames)

# Load filenames as Dask bag with db.read_text(): speeches
speeches = db.read_text(filenames)

# Print number of speeches with .count()
print(speeches.count().compute())



237


In [3]:


# Call .take(1): one_element
one_element = speeches.take(1)

# Extract first element of one_element: first_speech
first_speech = one_element[0]

# Print type of first_speech and first 60 characters
print(type(first_speech))
print(first_speech[:60])



<class 'str'>
 Fellow-Citizens of the Senate and House of Representatives:


In [4]:


# Call .str.split(' ') from speeches and assign it to by_word
by_word = speeches.str.split(' ')

# Map the len function over by_word and compute its mean
n_words = by_word.map(len)
avg_words = n_words.mean()

# Print the type of avg_words and value of avg_words.compute()
print(type(avg_words))
print(avg_words.compute())



<class 'dask.bag.core.Item'>
8239.084388185654


In [5]:


# Convert speeches to lower case: lower
lower = speeches.str.lower()

# Filter lower for the presence of 'health care': health
health = lower.filter(lambda s:'health care' in s)

# Count the number of entries : n_health
n_health = health.count()

# Compute and print the value of n_health
print(n_health.compute())




47


In [6]:


# Call db.read_text with congress/bills*.json: bills_text
bills_text=db.read_text('data/congress/bills*.json')

# Map the json.loads function over all elements: bills_dicts
bills_dicts=bills_text.map(json.loads)

# Extract the first element with .take(1) and index to the first position: first_bill
first_bill=bills_dicts.take(1)[0]

# Print the keys of first_bill
print(first_bill.keys())



dict_keys(['bill_resolution_type', 'bill_type', 'bill_type_label', 'committee_reports', 'congress', 'current_chamber', 'current_status', 'current_status_date', 'current_status_description', 'current_status_label', 'display_number', 'docs_house_gov_postdate', 'id', 'introduced_date', 'is_alive', 'is_current', 'link', 'lock_title', 'major_actions', 'noun', 'number', 'related_bills', 'scheduled_consideration_date', 'senate_floor_schedule_postdate', 'sliplawnum', 'sliplawpubpriv', 'source', 'source_link', 'sponsor', 'sponsor_role', 'text_incorporation', 'title', 'title_without_number', 'titles'])


In [7]:


# Compare the value of the 'current_status' key to 'enacted_veto_override'
def veto_override(d):
    return d['current_status'] == 'enacted_veto_override'



In [8]:

# Filter the bills: overridden
overridden = bills_dicts.filter(veto_override)

# Print the number of bills retained
print(overridden)

# Get the value of the 'title' key
titles = overridden.pluck('title')

# Compute and print the titles
print(titles.compute())



dask.bag<filter-..., npartitions=17>
['H.R. 12471 (93rd): A bill to amend section 552 of title 5, United States Code, known as the Freedom of Information Act.', 'H.R. 6198 (97th): A bill to amend the manufacturing clause of the copyright law.', 'H.R. 6863 (97th): Supplemental Appropriations Act, 1982']


In [9]:


# Define a function lifespan that takes a dictionary d as input
def lifespan(d):
    # Convert to datetime
    current = pd.to_datetime(d['current_status_date'])
    intro = pd.to_datetime(d['introduced_date'])

    # Return the number of days
    return (current - intro).days

# Filter bills_dicts: days
days = bills_dicts.filter(lambda s:s['current_status']=='enacted_signed').map(lifespan)

# Print the mean value of the days Bag
print(days.mean().compute())



228.03521703521704
